# 作用域 

## 作用域基础

每个变量名都有自己的命名空间，即作用域。变量的作用域在变量第一次赋值时决定，作用域决定了该变量的可见范围。

- def 内创建的变量只能在函数内使用，不能在函数外部引用该变量
- 函数内和函数外赋值的变量不冲突，即使名字相同

变量的作用域由它在代码中被赋值的位置决定：
- 在函数内赋值，则对于该函数而言是局部的
- 在外层函数定义，对内层函数而言是非局部的
- 在所有的函数外定义，则是全局的

## 作用域细节

- 外围模块是全局作用域
- 全局作用域的作用范围仅限于单个文件，python中没有跨文件的全局概念
- 函数内赋值的变量名除非声明为global或nonlocal,否则都是局部变量
- 除了函数内创建的变量名，其他使用到的变量都是外层函数的局部变量，全局变量或内置变量
- 函数每次调用都会创建一个新的局部作用域

函数内部任何类型的赋值都会把一个名称划定为局部，不仅限于=赋值，还包括import, def等

## 变量名解析：LEGB规则

所有函数内定义赋值的变量名都默认为局部变量。函数能够随意使用在外层函数内或全局作用域内的白能量，但是必须声明为非局部变量和全局变量来改变其属性。

当使用变量时，python会在四个作用域中找到第一次该变量名的地方停下来， 这就是LEGB规则：
- 首先是L，局部作用域
- 其次是E，外一层的局部作用域
- 之后是G，全局作用域
- 最后是B，内置作用域

当在函数中给一个变量名赋值时，会创建或改变局部作用域的变量名，除非该变量已被声明为全局的

## 作用域示例

In [6]:
X = 99 # 全局
def func(Y):
    X = 10 # 这里对X重新赋值，则生成的局部变量的X, 和外部的X不冲突，如果不重新赋值，则X是直接使用的全部X
    Z = X+Y
    return Z

print(func(1))
print(X)

11
99


上例中X和func是全局作用域变量，Y和Z则是局部的

## 内置作用域

内置作用域仅仅是一个名为`builtins`的内置模块，但是必须要导入`builtins`之后才能使用内置作用域。


In [8]:
import builtins
dir(builtins)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EnvironmentError',
 'Exception',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'RuntimeError',
 'RuntimeWarning',
 'StopAsyncIteration',
 'StopIter

In [9]:
zip is builtins.zip

True

如果对内置作用域的变量重新赋值，那么会替换到内置作用域的变量，这是万万不能的哦

## Global语句

global 语句告诉Python函数计划生成一个或多个全局变量名
- 全部变量是在外层模块文件顶层被赋值的变量名
- 全局变量如果在函数内赋值，必须经过声明
- 全局变量名在函数的内部不经声明也可以被引用

In [11]:
x = 88
def func():
    global x # 使用global声明后，可以在函数内引用全局变量，并重新赋值
    x = 99
func()
print(x)

99


In [13]:
y, z = 1, 2
def all_global():
    global x # 在函数内创建的变量，升级为全局变量
    x = y + z

all_global()
print(x) # 在函数外可以正常使用global声明的全局变量

3


**尽量避免使用全局变量**  
**尽量不要修改其他文件中的全局变量**

此外可以通过导入自身模块，模拟global
```python
# thismode.py 
var = 10

def func1():
    import thismode
    thismode.var += 1

def func2():
    import sys
    glob = sys.modules['thismode']
    glob.var += 1
```

## 作用域和嵌套函数

嵌套作用域举例

In [14]:
x = 99

def f1():
    x = 88
    def f2():
        print(x) # 这里访问的是嵌套作用域的x
    f2()

f1()

88


### 闭包

又叫工厂函数，能够记忆外层作用域的值，不管那些嵌套作用域是否还在内存中。

In [15]:
def maker(N):
    def action(X):
        return X**N 
    return action

外层函数简单地生成并返回一个嵌套地函数，却并不调用这个内嵌地函数。

In [16]:
f = maker(2)
f

<function __main__.maker.<locals>.action(X)>

f是一个内嵌函数的引用

In [18]:
f(3)

9

这里调用f即内层地action时，虽然maker()已经执行完毕并返回了值，但是参数N值却被action记忆了下来，每次调用f时，会计算平方  
**扁平胜于嵌套**

如果在函数中定义嵌套函数是在一个循环中，在这个嵌套函数又引用了一个外层变量，这个变量被循环改变，那么所有在这个循环中产生地函数都会有相同地值，就是最后一次循环的值

In [19]:
def makeAction():
    acts=[]
    for i in range(5):
        acts.append(lambda x:i**x) # 内嵌函数定义在循环中，引用地外部变量i地值每次循环都会改变
    return acts 

In [20]:
acts = makeAction()
print(acts[0](2))
print(acts[1](2)) # 这里的i每次的值都是循环最后一次的值

16
16


In [ ]:
def makeActions_new():
    acts=[]
    for i in range(5):
        acts.append(lambda x, i=i:i**x) # 将i当作参数传入
    return acts 

In [23]:
acts_new = makeActions_new()
print(acts_new[2](2))
print(acts_new[3](2))

4
9


## nolocal语句

使用`nolocal`声明后，内嵌地def就可以对外层函数中的名称进行读取和写入访问。nolocal语句和global语句很像，声明将要在外层作用域中修改的名称。和global的不同之处在于，nolocal作用于外层函数的作用域中的一个名称，而不是所有def之外的全局模块作用域。声明nolocal名称的时候，它必须已经存在于该外层函数的作用域中。

### 基础

nolocal语句允许内嵌函数修改定义在语法上位于外层的函数的作用域中的一个或多个名称。nolocal语句除了允许修改外层def中的名称外，还会强制从外层作用域查找引用的变量，而忽略局部作用域。

In [3]:
def tester(start):
    state = start 
    def nested(label):
        nonlocal state
        print(label, state)
        state += 1 # 声明为nolocal后，在内嵌函数中可以修改外层函数中变量
    return nested

In [4]:
F = tester(0)

In [5]:
F('szq')

szq 0


In [6]:
F('lqf') # 可见state的值可以被修改了

lqf 1


In [7]:
def tester_new(start):
    state = start 
    def nested(label):
        print(label, state)
        state += 1 # 不适用nolocal会报错的
    return nested

In [8]:
F = tester_new(0)
F('szq')

UnboundLocalError: local variable 'state' referenced before assignment

In [9]:
def tester_1(start):
    def nested(label):
        nonlocal state # 必须在外层中声明过才行
        state = 0
        print(label, state)
    return nested

SyntaxError: no binding for nonlocal 'state' found (<ipython-input-9-f79df300278b>, line 3)

只有变量必须改变时我们才能将其声明为nolocal，而且nolocal名称在外层函数外面时不可见的。